In [1]:
!pip install boto
!pip install git+git://github.com/met-office-lab/asn_data_utils.git

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Cloning git://github.com/met-office-lab/asn_data_utils.git to /tmp/pip-u1qlw9nm-build
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import iris
from asn_utils.Loader import Loader
from functools import reduce
from collections import Counter
import numpy as np

In [2]:
myloader = Loader()
fs = myloader.list_files("mogreps")[:5]

In [3]:
fs

['/usr/local/share/notebooks/data/mogreps/201612/prods_op_mogreps-g_20161203_00_00_003.pp',
 '/usr/local/share/notebooks/data/mogreps/201612/prods_op_mogreps-g_20161203_00_00_006.pp',
 '/usr/local/share/notebooks/data/mogreps/201612/prods_op_mogreps-g_20161203_00_00_009.pp',
 '/usr/local/share/notebooks/data/mogreps/201612/prods_op_mogreps-g_20161203_00_00_012.pp',
 '/usr/local/share/notebooks/data/mogreps/201612/prods_op_mogreps-g_20161203_00_00_015.pp']

In [4]:
draw = iris.load_raw(fs[0], 'wet_bulb_potential_temperature')

In [5]:
print(draw)

0: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)
1: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)
2: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)
3: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)
4: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)
5: wet_bulb_potential_temperature / (K) (latitude: 600; longitude: 800)


In [6]:
merged = draw.merge_cube()

In [7]:
print(merged)

wet_bulb_potential_temperature / (K) (time: 2; pressure: 3; latitude: 600; longitude: 800)
     Dimension coordinates:
          time                            x            -            -               -
          pressure                        -            x            -               -
          latitude                        -            -            x               -
          longitude                       -            -            -               x
     Auxiliary coordinates:
          forecast_period                 x            -            -               -
     Scalar coordinates:
          forecast_reference_time: 2016-12-03 00:00:00
     Attributes:
          STASH: m01s16i205
          source: Data from Met Office Unified Model
          um_version: 10.4


In [8]:
subd = draw[:-1]

In [9]:
subd.merge()

[<iris 'Cube' of wet_bulb_potential_temperature / (K) (-- : 5; latitude: 600; longitude: 800)>]

In [10]:
acoord = merged.coord('pressure')

In [11]:
# def magic_subset_merge(cube_list, DimCoords):
#     return extract_overlapping(cube_list, DimCoords).merge_cube()

In [12]:
#magic_subset_merge(subd, ["latitude", "longitude", "time", "pressure"])

In [16]:
draw

lon_0 = draw[0].coord('longitude')
lat_0 = draw[0].coord('latitude')
lat_1 = draw[1].coord('latitude')
lat_2 = draw[2].coord('latitude')
pres_0 = draw[0].coord('pressure')
pres_1 = draw[1].coord('pressure')
lat_3 = draw[3].coord('latitude')
lon_1 = draw[1].coord('longitude')


as_list = [lon_0, lat_0, lat_1, lat_2, pres_0, pres_1, lat_3, lon_1]
as_list_rep = [0,1,1,1,2,3,1,0]
as_set = set(as_list)

len(as_set)

c = Counter(as_list)

len(c) #This shows that set and collections.Counter both use __hash__ and not __equ__ for compairing items.

print(pres_0 == pres_1)

False


In [67]:
def sort_and_unique_cells(items):
    sorted_items = sorted(items, key=attrgetter('val'))
    return [sorted_items[0]] + [sorted_items[i + 1] for i in range(len(sorted_items) -1) if not sorted_items[i + 1] == sorted_items[i] ]

def get_unique_dims(all_dims):
    """
    Takes a list of dims and returns a dict:
    
    {dim:count}
    
    """
    all_dims = np.array(all_dims, dtype=object)
    i = 0
    dim_set = {}
    while i < len(all_dims):
        
        dim = all_dims[i]
        
        if any(dim == other_dim for other_dim in dim_set.keys()): 
            i += 1 # If we've seen this dim before contiune
            continue
        else:
            dim_set[dim] = 0 # Else add it to our set. Add it with count 0 as we will perform the count next.
            
        matches = (dim == all_dims)
        dim_set[dim] = np.where(matches)[0].shape[0] # Count the occurances of this dimention. 
        # Looking forward in the list of dimentions are there any other dims that don't match
        idx_of_forward_non_matches = np.where(matches[i:] == False)[0] 
        print('--',idx_of_forward_non_matches) # indexes of non matches with current dim offset by current posision. 
        
        
        if not len(idx_of_forward_non_matches):
            break; # We've found all dims. Break.
            
        # If there are non-matches continue our search from the first location of a non match.
        i = (idx_of_forward_non_matches[0] + i)
    return dim_set
    
print(as_list_rep)

(count_unique_dims(as_list)).values()

[0, 1, 1, 1, 2, 3, 1, 0]
-- [1 2 3 4 5 6]
-- [3 4 6]
-- [1 2 3]
-- [1 2]


dict_values([2, 4, 1, 1])

In [63]:
"""class iris.coords.Coord(points, standard_name=None, long_name=None, var_name=None, units='1', bounds=None, attributes=None, coord_system=None)
"""
p0 = draw[0].coord('pressure')
p1 = draw[1].coord('pressure')

#iris.coords.DimCoord(list(p0.cells()) + list(p0.cells()))

list(p0.cells())+ list(p1.cells())
list(p0.cells())[0].bound


dims = [p0, p1]

def dim_list_to_single_dim(dim_list):
    def to_points_and_bounds(list_of_dims):
        points, bounds = list(zip(*[(item.point, item.bound) for sublist in [list(p.cells()) for p in list_of_dims] for item in sublist]))
        return points, bounds

    eg_dim = dim_list[0]
    points, bounds = to_points_and_bounds(dim_list)
    bounds = bounds if all(bounds) else None
    new_dim = iris.coords.DimCoord(points=points, bounds=bounds,
                         standard_name=eg_dim.standard_name, units=eg_dim.units,
                         long_name=eg_dim.long_name, attributes=eg_dim.attributes,
                         coord_system=eg_dim.coord_system, circular = eg_dim.circular,
                        var_name = eg_dim.var_name)
    return new_dim

dim_list_to_single_dim(dims)

DimCoord(array([ 850.,  500.], dtype=float32), standard_name=None, units=Unit('hPa'), long_name='pressure')

This shows that `set` and `collections.Counter` both use `__hash__` and not `__equ__` for compairing items.

In [ ]:

cds = [acoord[0], acoord[0], acoord[1]]

In [18]:
sort(cds, key=lambda a, b: a.points[0] > b.points[0])

NameError: name 'sort' is not defined

In [77]:
 def extract_overlapping(cl, coord_names):
        """
        Returns a :class:`CubeList` of cubes extracted over regions
        where they coordinates overlap, for the coordinates
        in coord_names

        Args:

        * coord_names:
           A string or list of strings of the names of the coordinates
           over which to perform the extraction.

        """
        print(".")
        if type(coord_names) is str:
            coord_names = [coord_names]

        def make_overlap_fn(coord_name):
            def overlap_fn(cell):
                return all(cell in cube.coord(coord_name).cells()
                            for cube in cl)
            return overlap_fn
 
        
        new_coords = []
        for coord_name in coord_names:
            uniq_dims = get_unique_dims([cube.coord(coord_name) for cube in cl])
            #coords_are_equal = all(cl[0].coord(coord_name) == cube.coord(coord_name) for cube in cl)
            if len(uniq_dims) == 0:
                print(coord_name)
                new_coords.append(uniq_dims.keys()[0])
            else:
                dims = list(uniq_dims.keys())
                as_sigle_dim = dim_list_to_single_dim(dims)
                new_coords.append(as_sigle_dim)
        return new_coords

In [78]:
extract_overlapping(subd, ["latitude", "longitude", "time", "pressure"])

.
-- []
-- []
-- [3 4]
-- []
-- [1 2 4]
-- [1 2]
-- [1 2]


[DimCoord(array([-89.84999847, -89.54999542, -89.25      , -88.94999695,
        -88.65000153, -88.34999847, -88.04999542, -87.75      ,
        -87.44999695, -87.15000153, -86.84999847, -86.54999542,
        -86.25      , -85.94999695, -85.65000153, -85.34999847,
        -85.04999542, -84.75      , -84.44999695, -84.15000153,
        -83.84999847, -83.54999542, -83.25      , -82.94999695,
        -82.65000153, -82.34999847, -82.04999542, -81.75      ,
        -81.44999695, -81.1499939 , -80.84999847, -80.54999542,
        -80.25      , -79.94999695, -79.6499939 , -79.34999847,
        -79.04999542, -78.75      , -78.44999695, -78.1499939 ,
        -77.84999847, -77.54999542, -77.25      , -76.94999695,
        -76.6499939 , -76.34999847, -76.04999542, -75.75      ,
        -75.44999695, -75.1499939 , -74.84999847, -74.54999542,
        -74.25      , -73.94999695, -73.6499939 , -73.34999847,
        -73.04999542, -72.75      , -72.44999695, -72.1499939 ,
        -71.84999847, -71.54999

In [76]:
keys = list({'a':'A'}.keys())
keys[0]


'a'